<a href="https://colab.research.google.com/github/sudhanshu2k/Sarcasm/blob/master/TestEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [10]:
cd Sarcasm

/content/drive/My Drive/Sarcasm


In [11]:
ls

'='/                               __pycache__/
 cache/                            pytorch_model.bin
 config.json                       Sarcasm_dataset_without_label.csv
 convert_examples_to_features.py   Sarcasm.tar.gz
 Data_Comment/                     TestData/
 data_unbalanced/                  test.tsv
 eval_features.pkl                 tools.py
 eval_results.txt                  train_comment.tsv
 eval_test_features.pkl            train_features.pkl
 output/                           train.tsv
 output1/                          vocab.txt
 output_unbalanced/


In [12]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 34.9MB/s 
     |████████████████████████████████| 675kB 40.8MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609171 sha256=e1f80f77c3415a367e94e0ab30e674b79ab45a5aec8ac500ef80bda0aadcee22
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built regex


In [0]:
import torch
import numpy as np
import pickle

from sklearn.metrics import matthews_corrcoef, confusion_matrix

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from torch.nn import CrossEntropyLoss, MSELoss

from tools import *
from multiprocessing import Pool, cpu_count
import convert_examples_to_features

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving Sarcasm_dataset_without_label.csv to Sarcasm_dataset_without_label.csv


In [0]:
DATA_DIR='TestData/'
# Bert pre-trained model selected in the list: bert-base-uncased,
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'output1/Sarcasm1.tar.gz'
# The name of the task to train.
TASK_NAME = 'Sarcasm'
# The output directory where the model predictions and checkpoints will be written.
OUTPUT_DIR = 'output1/'
# The maximum total input sequence length after WordPiece tokenization.

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128
TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'
CONFIG_NAME = "output1/SarcasmModel/bert_config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:

# import data
import pandas as pd
test=pd.read_csv('Sarcasm_dataset_without_label.csv')

In [0]:
test['Comment']=test['Textual data for sarcasm detection']

In [25]:
test['Comment'].head(5)

0    passengers praying uber just a hobby for elder...
1            the top 10 workout songs for january 2018
2    meryl streep looks exactly like the 'shrek' fa...
3    couple just wants small ceremony in public par...
4    terrified fda warns something making bananas b...
Name: Comment, dtype: object

In [26]:
#Lower Case
test['Comment'] = test['Comment'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Comment'].head()

0    passengers praying uber just a hobby for elder...
1            the top 10 workout songs for january 2018
2    meryl streep looks exactly like the 'shrek' fa...
3    couple just wants small ceremony in public par...
4    terrified fda warns something making bananas b...
Name: Comment, dtype: object

In [27]:
#Removing Punctuation
test['Comment'] = test['Comment'].str.replace('[^\w\s]','')
print('{} words.'.format(len([word for sentence in test['Comment'] for word in str(sentence).split()])))

2045 words.


In [28]:
#Removal of Stop Words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
test['Comment'] = test['Comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
print('{} words.'.format(len([word for sentence in test['Comment'] for word in str(sentence).split()])))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
1489 words.


In [29]:
#Lametization
import nltk
nltk.download('wordnet')
from textblob import Word
test['Comment'] = test['Comment'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print('{} words.'.format(len([word for sentence in test['Comment'] for word in str(sentence).split()])))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
1489 words.


In [30]:
#Removal of Number
test['Comment'] = test['Comment'].str.replace('\d+','')
print('{} words.'.format(len([word for sentence in test['Comment'] for word in str(sentence).split()])))

1468 words.


In [0]:
test['dummy_1'] = range(0,len(test['Comment']))
test['dummy_2'] = '*'
test['label']=0
test=test[['dummy_1','label','dummy_2','Comment']]

In [0]:
test.to_csv('TestData/test.tsv', sep='\t', index=False, header=False)

In [0]:
!rm TestData/test.tsv

In [0]:
!cp TestData/test.tsv TestData/dev.tsv

In [18]:
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file output1/vocab.txt


In [0]:
processor = BinaryClassificationProcessor()
eval_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

In [21]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

Preparing to convert 201 examples..
Spawning 1 processes..


In [0]:
with open("eval_test_features.pkl", 'wb') as f:
    pickle.dump(eval_features, f)

In [0]:
with open("eval_test_features.pkl", "rb") as f:
     eval_features = pickle.load(f)

In [0]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

In [0]:
if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)

In [0]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [31]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=len(label_list))

INFO:pytorch_pretrained_bert.modeling:loading archive file output1/Sarcasm1.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file output1/Sarcasm1.tar.gz to temp dir /tmp/tmpgu7z9cks
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



In [32]:
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [65]:
model.eval()
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)
preds = preds[0]

In [68]:
print(len(preds))

201


In [0]:
preds_1 = np.argmax(preds, axis=1)

In [71]:
print(preds_1)

[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 1 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0
 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 1 1 0 1
 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 0 0 1
 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 1 0 1
 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0]
